In [7]:
import pandas as pd

In [43]:
who = pd.read_csv("https://covid19.who.int/WHO-COVID-19-global-data.csv", delim_whitespace=True)
who = who.set_index(["Date_reported", " Country_code"])
who.head(100)

,,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
Date_reported,Country_code,,,,,,
2020-02-24,AF,Afghanistan,EMRO,5,5,0,0
2020-02-25,AF,Afghanistan,EMRO,0,5,0,0
2020-02-26,AF,Afghanistan,EMRO,0,5,0,0
2020-02-27,AF,Afghanistan,EMRO,0,5,0,0
2020-02-28,AF,Afghanistan,EMRO,0,5,0,0
...,...,...,...,...,...,...,...
2020-05-29,AF,Afghanistan,EMRO,643,13659,11,246
2020-05-30,AF,Afghanistan,EMRO,784,14443,2,248
2020-05-31,AF,Afghanistan,EMRO,651,15094,2,250


In [30]:
country_codes = pd.read_csv("https://datahub.io/core/country-codes/r/country-codes.csv")
country_codes = country_codes.set_index("ISO3166-1-Alpha-2")
country_codes = country_codes[["ISO3166-1-Alpha-3"]]
country_codes.head()

,ISO3166-1-Alpha-3
ISO3166-1-Alpha-2,
TW,TWN
AF,AFG
AL,ALB
DZ,DZA
AS,ASM


In [28]:
from io import BytesIO
from zipfile import ZipFile
import urllib.request

population_url = "http://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=csv"
request = urllib.request.urlopen(population_url)
populations = None

with ZipFile(BytesIO(request.read())) as zip_file:
    for contained_file in zip_file.namelist():
        if not contained_file.startswith("Metadata"):
            print(contained_file)
            populations = pd.read_csv(zip_file.open(contained_file), skiprows=4)

populations = populations.set_index("Country Code")
populations[["2019"]].head()

API_SP.POP.TOTL_DS2_en_csv_v2_1308146.csv


,2019
Country Code,
ABW,106314.0
AFG,38041754.0
AGO,31825295.0
ALB,2854191.0
AND,77142.0
